In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import numpy as np
import time
import random
import math
import copy
from matplotlib import pyplot as plt

In [2]:
import ofa
import ofa.model_zoo
import ofa.tutorial
import ofa.nas.accuracy_predictor.arch_encoder
import ofa.nas.search_algorithm
import ofa.nas.efficiency_predictor.latency_lookup_table

In [3]:
random_seed = 1
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
print('Successfully imported all packages and configured random seed to %d!'%random_seed)

cuda_available = torch.cuda.is_available()
if cuda_available:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(random_seed)
    print('Using GPU.')
else:
    print('Using CPU.')

Successfully imported all packages and configured random seed to 1!
Using GPU.


In [4]:
ofa_network = ofa.model_zoo.ofa_net('ofa_mbv3_d234_e346_k357_w1.2', pretrained=True)
#ofa_network = ofa.model_zoo.ofa_net('ofa_resnet50', pretrained=True)
print('The OFA Network is ready.')

The OFA Network is ready.


In [5]:
target_hardware = 'note10'
latency_table = ofa.tutorial.LatencyTable(device=target_hardware)
print('The Latency lookup table on %s is ready!' % target_hardware)

Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/160_lookup_table.yaml" to C:\Users\andy/.hancai/latency_tools/160_lookup_table.yaml
C:\School\CS8803-Project\ofa\tutorial\latency_table.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.lut = yaml.load(fp)
Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/176_lookup_table.yaml" to C:\Users\andy/.hancai/latency_tools/176_lookup_table.yaml


Built latency table for image size: 160.
Built latency table for image size: 176.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/192_lookup_table.yaml" to C:\Users\andy/.hancai/latency_tools/192_lookup_table.yaml


Built latency table for image size: 192.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/208_lookup_table.yaml" to C:\Users\andy/.hancai/latency_tools/208_lookup_table.yaml


Built latency table for image size: 208.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/224_lookup_table.yaml" to C:\Users\andy/.hancai/latency_tools/224_lookup_table.yaml


Built latency table for image size: 224.
The Latency lookup table on note10 is ready!


In [6]:
if cuda_available:
    # path to the ImageNet dataset
    print("Please input the path to the ImageNet dataset.\n")
    imagenet_data_path = input()
    #imagenet_data_path = 'C:\School\once-for-all-master\imgnet'

    # if 'imagenet_data_path' is empty, download a subset of ImageNet containing 2000 images (~250M) for test
    if not os.path.isdir(imagenet_data_path):
        os.makedirs(imagenet_data_path, exist_ok=True)
        ofa.utils.download_url('https://hanlab.mit.edu/files/OnceForAll/ofa_cvpr_tutorial/imagenet_1k.zip', model_dir='data')
        ! cd data && unzip imagenet_1k 1>/dev/null && cd ..
        ! copy -r data/imagenet_1k/* $imagenet_data_path
        ! del -rf data
        print('%s is empty. Download a subset of ImageNet for test.' % imagenet_data_path)

    print('The ImageNet dataset files are ready.')
else:
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

Please input the path to the ImageNet dataset.

C:\School\CS8803-Project\imgnet
The ImageNet dataset files are ready.


In [7]:
if cuda_available:
    # The following function build the data transforms for test
    def build_val_transform(size):
        return transforms.Compose([
            transforms.Resize(int(math.ceil(size / 0.875))),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])

    data_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(
            root=os.path.join(imagenet_data_path, 'val'),
            transform=build_val_transform(224)
        ),
        batch_size=250,  # test batch size
        shuffle=True,
        num_workers=16,  # number of workers for the data loader
        pin_memory=True,
        drop_last=False,
    )
    print('The ImageNet dataloader is ready.')
else:
    data_loader = None
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

The ImageNet dataloader is ready.


In [8]:
#accuracy_predictor = ofa.nas.accuracy_predictor.AccuracyPredictor(
accuracy_predictor = ofa.tutorial.AccuracyPredictor(
    pretrained=True,
    device='cuda:0' if cuda_available else 'cpu'
)

print('The accuracy predictor is ready!')

The accuracy predictor is ready!


Note: search time currently inaccurate

In [9]:
latency_constraint = (20,25,30)  # ms, suggested range [15, 33] ms
P = 100  # The size of population in each generation
N = 500  # How many generations of population to be searched
N2 = 100
r = 0.25  # The ratio of networks that are used as parents for next generation
params = {
    'constraint_type': target_hardware, # Let's do FLOPs-constrained search
    'efficiency_constraint': latency_constraint,
    'mutate_prob': 0.1, # The probability of mutation in evolutionary search
    'mutation_ratio': 0.5, # The ratio of networks that are generated through mutation in generation n >= 2.
    'efficiency_predictor': latency_table, # To use a predefined efficiency predictor.
    'accuracy_predictor': accuracy_predictor, # To use a predefined accuracy_predictor predictor.
    'population_size': P,
    'max_time_budget': N,
    'max_time_budget2': N2,
    'parent_ratio': r,
}

# build the evolution finder
finder = ofa.tutorial.EvolutionFinder(**params)

# start searching
result_lis = []
st = time.time()
best_valids, best_info = finder.run_evolution_search_multi()
for i in range(len(latency_constraint)):
    result_lis.append(best_info[i])
    ed = time.time()
    print('Found best architecture on %s with latency <= %.2f ms in %.2f seconds! '
          'It achieves %.2f%s predicted accuracy with %.2f ms latency on %s.' %
          (target_hardware, latency_constraint[i], ed-st, best_info[i][0] * 100, '%', best_info[i][-1], target_hardware))

    # visualize the architecture of the searched sub-net
    _, net_config, latency = best_info[i]
    ofa_network.set_active_subnet(ks=net_config['ks'], d=net_config['d'], e=net_config['e'])
    print('Architecture of the searched sub-net:')
    print(ofa_network.module_str)

Searching with note10 constraint (30): 100%|█████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 30.67it/s]

Found best architecture on note10 with latency <= 20.00 ms in 25.95 seconds! It achieves 80.64% predicted accuracy with 19.82 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E6.0, K3), Identity)
(SE(O48, E4.0, K3), Identity)
((O96, E3.0, K7), None)
((O96, E4.0, K5), Identity)
((O96, E3.0, K3), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K7), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E4.0, K3), None)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E4.0, K3), Identity)
(SE(O192, E3.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear

Found best architecture on note10 with latency <= 25.00 ms in 25.95 seconds! It achieves 82.03% predicted accuracy with 24.94 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K